In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import math
import pickle 
path = Path('./')
train_path = path.joinpath('train.csv')
test_path = path.joinpath('test.csv')
submission_path = path.joinpath('submission_popular.csv')
metadata_path = path.joinpath('item_metadata.csv')

In [2]:
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
df_meta = pd.read_csv(metadata_path)
df_submission = pd.read_csv(submission_path)

In [3]:
action_list = set(df_train['action_type'].values)
reference_list = set(df_train['reference'].values)
platform_list = set(df_train['platform'].values)
city_list = set(df_train['city'].values)
device_list = set(df_train['device'].values)
filter_list = set(df_train['current_filters'].values)
filter_list =  [x for x in filter_list if str(x) != 'nan']

action_list = list(action_list)
action_list.sort()

reference_list = list(reference_list)
reference_list.sort()

platform_list = list(platform_list)
platform_list.sort()

city_list = list(city_list)
city_list.sort()

device_list = list(device_list)
device_list.sort()

filter_list = list(filter_list)
filter_list.sort()

In [ ]:
#reference embedding 보조 함수
items = df_meta['item_id'].values
properties = df_meta['properties'].values
all_items_test = []
for i in range(len(properties)):
    p_data = properties[i].split('|')
    all_items_test += p_data
set_items = set(all_items_test)

list_items = list(set_items)
list_items.sort()
def embedding_item_diction():
    all_items = {}
    for i in range(len(properties)):
        all_items[items[i]] = properties[i].split('|')
    embedding_items = {}
    #print(len(list_items))
    for keys in all_items.keys():
        item_embeddings = [0]*len(list_items)
        for i in range(len(list_items)):
            if list_items[i] in all_items[keys]:
                item_embeddings[i] = 1
        embedding_items[keys] = item_embeddings
    return embedding_items
reference_diction = embedding_item_diction()

In [4]:
#테스트 데이터에서 마지막 reference가 nan인지 확인
def check_data(data):
    max_error = 0
    datalength = len(data)
    for i in range(datalength):
        if(isinstance(data[i][-1,5], float)):
            if(math.isnan(data[i][-1,5]) == True):
                pass
            else:
                print('error')
                print(data[i])
                max_error +=1
        else:
            print('error')
            print(data[i])
            max_error +=1
        if(max_error > 10):
            break

In [5]:
#테스트 데이터에서 reference가 nan일때 그 전 action이 click out인지 확인
def check_data_alpha(data):
    max_error = 0
    datalength = len(data)
    for i in range(datalength):
        for j in range(len(data[i])):
            if(isinstance(data[i][j,5], float)):
                if(math.isnan(data[i][j,5]) == True):
                     if(data[i][j,4] != 'clickout item'):
                        print(data[i])
                        max_error +=1
        
        if(max_error > 10):
            break

In [6]:
#테스트 데이터에서 reference에 하나라도 nan이 있는지 판별 있다면 그 중 적어도 하나는 action이 click out인지 판별
def check_data_beta(data):
    max_error = 0
    datalength = len(data)
    for i in range(datalength):
        error_data_number = 0
        click_check = False
        for j in range(len(data[i])):
            if(isinstance(data[i][j,5], float) and math.isnan(data[i][j,5]) == True):
                error_data_number += 1
        if(error_data_number == 0):
            print(data[i])
            print('error')
            max_error +=1
        if(error_data_number != 0):
            for j in range(len(data[i])):
                if(isinstance(data[i][j,5], float) and math.isnan(data[i][j,5]) == True):
                    if(data[i][j,4] == 'clickout item'):
                        click_check = True
        if(click_check == False):
            print('error')
            print(data[i])
            max_error +=1
            #print(data[i])
            #print('datas')
        if(max_error > 10):
            break

In [66]:
def check_inference(data):
    datalength = len(data)
    maxlength = 0
    correct_data = 0
    new_data = []
    for i in range(datalength):
        if(data[i][-1,5] in data[i][-1,10].split('|')):
            if int(data[i][-1,5]) in reference_diction:
                new_data.append(data[i])
                correct_data += 1
        else:
            pass
            #print(i)
            #print('error')
            #maxlength +=1
    print(correct_data)
    return new_data

In [67]:
# 트레이닝 데이터를 세션 단위로 나눔
def make_data(data): 
    data_split = []
    temp_data = []
    train_size = len(data)
    action_index = 1
    for i in range(train_size):
        if(i==0):
            temp_data.append(data[i])
        if(i>0):
            if(data[i,action_index] == data[i-1,action_index]):
                temp_data.append(data[i])
            else:
                data_split.append(np.array(temp_data))
                temp_data = []
                temp_data.append(data[i])
    return data_split

In [61]:
train_data_split = make_data(df_train.values)

In [62]:
#트레이닝 데이터에서 click out이 존재하는 데이터만 뽑아냄
#트레이닝 데이터에서 끝부분이 click out이 되도록 뒷부분을 자름
def data_preprocess(data): 
    new_data = []
    for i in range(len(data)):
        check_data = False
        final_click = 0
        for j in range(len(data[i])):
            if(data[i][j,4] =='clickout item'):
                final_click = j
                check_data = True
        if(final_click < len(data[i])-1 and check_data):
            new_data.append(np.delete(data[i],np.s_[final_click+1:],0))
        elif(check_data):
            new_data.append(data[i])
    return new_data

In [68]:
train_data_preprocess = data_preprocess(train_data_split)

In [69]:
len(train_data_preprocess) #총 데이터 개수

826874

In [70]:
train_data_preprocess = check_inference(train_data_preprocess)

826199


In [71]:
def RepresentsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

In [72]:
len(reference_diction)

927142

In [73]:
len(reference_diction[1257342])

157

In [19]:
#filter embedding 보조함수
all_filters = []
for i in range(len(filter_list)):
    if i>0:
        all_filters += filter_list[i].split('|')
all_filter_set = set(all_filters)
all_filter_list = list(all_filter_set)
all_filter_list.sort()
def filter_embedding(data, version):
    embedding_data = [0]*202
    if version == 1:
        datalist = data.split('|')
        for filters in datalist:
            embedding_data[all_filter_list.index(filters)] = 1
        return embedding_data

In [20]:
def one_hot_encoding(number, length):
    return [int(i==number) for i in range(length)]

In [21]:
def sum_list(list1, list2):
    length = len(list1)
    final_list = []
    for i in range(length):
        final_list.append(list1[i] +list2[i])
    return final_list

In [22]:
type(city_list)

list

In [23]:
# 전체 action embedding
def embedding(data, action_index):
    if action_index == 0: #user_id dim 0
        return []
    
    if action_index == 1: # session_id dim 0
        return []
    
    if action_index == 2: # timestep dim 0
        return []
    
    if action_index == 3: #step dim 1
        return []
    
    if action_index == 4: #action_type dim 10
        #print(one_hot_encoding(action_list.index(data), 10))
        return one_hot_encoding(action_list.index(data), 10)
    
    if action_index == 5: #reference dim 157
        if RepresentsInt(data):
            return reference_diction[int(data)]
        else:
            return [0]*157
        
    if action_index == 6: #platform dim 55
        return one_hot_encoding(platform_list.index(data), 55)
    
    if action_index == 7: #city dim 0
        #return city_list.index(data)
        return []
    
    if action_index == 8: #device dim 3
        return one_hot_encoding(device_list.index(data), 3)
    
    if action_index == 9: #current_filters dim 202
        if(isinstance(data, float) and math.isnan(data) == True):
            return [0]*202
        else:
            return filter_embedding(data, 1)
        
    if action_index == 10: #impressions dim 157
        if(isinstance(data, float) and math.isnan(data) == True):
            return [0]*157
        else:
            impression_embedding = [0]*157
            impression_list = data.split('|')
            for impression in impression_list:
                try:
                    impression_embedding =sum_list(impression_embedding, reference_diction[int(impression)])
                    
                except:
                    pass
                    #print(int(impression))
            '''
            if(len(impression_embedding) < 3925):
                impression_embedding += [0]*(3925-len(impression_embedding))   
            '''
            return impression_embedding
            
        
    if action_index == 11: #prices dim 25
        if(isinstance(data, float) and math.isnan(data) == True):
            return [0]*25
        else:
            price_embedding = []
            for prices in data.split('|'):
                price_embedding.append(int(prices))
            if(len(price_embedding) < 25):
                price_embedding += [0]*(25-len(price_embedding))   
            return price_embedding

In [24]:
def check_last_click(data):
    data_number = 0
    for i in range(len(data)):
        if(data[i][-1,4] == 'clickout item'):
            data_number +=1
    return data_number

In [25]:
action_type_length = 10
reference_length = 157
platform_length = 55
device_length = 3
current_filters_length = 202
impresssions_length = 3925
prices_length = 25
def addVector(vector1, vector2, coef, action_num):
    new_action_type = []
    new_reference = []
    new_platform = []
    new_device = []
    new_filters = []
    new_impressions = []
    new_prices = [] 
    #print(len(vector1))
    #print(len(vector2))
    for i in range(10):
        new_action_type.append(vector1[i] + np.power(coef,action_num)*vector2[i])
    
    for i in range(10, 167):
        new_action_type.append(vector1[i] + np.power(coef,action_num)*vector2[i])
        
    for i in range(167, 222):
        new_action_type.append(vector1[i])
    
    for i in range(222, 225):
        new_action_type.append(vector1[i])
        
    for i in range(225, 427):
        new_action_type.append(vector1[i] + np.power(coef,action_num)*vector2[i])
        
    return new_action_type

In [26]:
def MakeEmbeddingVector1(data):
    #embedding_vector = []
    #temp_vector = []
    session_vector = [0]*609 # session data embedding(length 10+157+55+3+202+157+25=609)
    item_vector = [] # item data embedding
    num_action = len(data)
    time_delay_coef = 0.1

    for i in range(num_action-1, -1, -1): #거꾸로
        timestep_vector = []
        for j in range(12):
            timestep_vector += embedding(data[i,j], j) 
        if(i == num_action-1):
            for idx in range(10, 167):
                timestep_vector[idx] = 0 # reference 부분을 0으로 만듦
        session_vector = addVector(session_vector, timestep_vector, time_delay_coef, num_action-1-i) #session vector 생성   
    item_vector.append(reference_diction[int(data[-1,5])])
    temp_impressions = data[-1, 10].split('|')
    for i in range(len(temp_impressions)):

        try:
            if(data[-1,5] != temp_impressions[i]):
                item_vector.append(reference_diction[int(temp_impressions[i])])
                #item_vector.append(int(temp_impressions[i]))
        except:
            #print('error')
            pass
    return session_vector, item_vector

In [27]:
def MakeEmbeddingVector2(data):
    #embedding_vector = []
    #temp_vector = []
    session_vector = [0]*609 # session data embedding(length 10+157+55+3+202+157+25=609)
    item_vector = [] # item data embedding
    num_action = len(data)
    time_delay_coef = 0.1
    correct_index = []
    for i in range(num_action-1, -1, -1): #거꾸로
        timestep_vector = []
        for j in range(12):
            timestep_vector += embedding(data[i,j], j) 
        if(i == num_action-1):
            for idx in range(10, 167):
                timestep_vector[idx] = 0 # reference 부분을 0으로 만듦
        session_vector = addVector(session_vector, timestep_vector, time_delay_coef, num_action-1-i) #session vector 생성   
    #item_vector.append(reference_diction[int(data[-1,5])])
    temp_impressions = data[-1, 10].split('|')
    for i in range(len(temp_impressions)):
        if(data[-1,5] == temp_impressions[i]):
            correct_index.append(i)
        item_vector.append(reference_diction[int(temp_impressions[i])])
    return session_vector, item_vector, correct_index

In [109]:
def MakeEmbeddingVector3(data):
    #embedding_vector = []
    #temp_vector = []
    #session_vector = [0]*609 # session data embedding(length 10+157+55+3+202+157+25=609)
    item_vector = [] # item data embedding
    num_action = len(data)
    #time_delay_coef = 0.1
    correct_index = []
    reference_vector = []
    filter_vector = [0]*202
    current_reference  = ''
    reference_size = 0
    for i in range(num_action-1, -1, -1): #거꾸로
        timestep_vector = []
        if i != num_action-1 and RepresentsInt(data[i, 5]) and current_reference != data[i,5] and reference_size < 2:
            current_reference = data[1,5]
            if int(data[i, 5]) in reference_diction:
                reference_size += 1
                reference_vector += reference_diction[int(data[i, 5])]
        if(filter_vector == [0]*202):
            filter_vector = embedding(data[i,9], 9)
    reference_vector += [0]*(314-len(reference_vector))
    session_vector = embedding(data[-1, 4], 4) + reference_vector + embedding(data[-1, 6], 6) + embedding(data[-1, 8], 8) + filter_vector
    # 10+314+202+55+3 = 584
    '''
    for j in range(12):
        timestep_vector += embedding(data[i,j], j) 

    if(i == num_action-1):
        for idx in range(10, 167):
            timestep_vector[idx] = 0 # reference 부분을 0으로 만듦
    session_vector = addVector(session_vector, timestep_vector, time_delay_coef, num_action-1-i) #session vector 생성
    '''
    #item_vector.append(reference_diction[int(data[-1,5])])
    temp_impressions = data[-1, 10].split('|')
    for i in range(len(temp_impressions)):
        if(data[-1,5] == temp_impressions[i]):
            correct_index.append(i)
        if int(temp_impressions[i]) in reference_diction:
            item_vector.append(reference_diction[int(temp_impressions[i])])
        else:
            item_vector.append([0]*157)
    return session_vector, item_vector, correct_index

In [75]:
x = [0,0,0]
x[0:3] =[1,1,1]
x + [0]*(3-3)

[1, 1, 1]

In [76]:
sevec, itemvec, crindex = MakeEmbeddingVector3(train_data_preprocess[0])

In [77]:
len(itemvec)

25

In [78]:
del df_train
del df_test
del df_meta
del df_submission
del train_data_split

final_train_data = []
final_train_label = []
datalength = len(train_data_preprocess)

num_error = 0
for i in range(datalength):
    if(i%10000 ==0):
        print('making data....', i)
    try:
        temp_train, temp_label = MakeEmbeddingVector(train_data_preprocess[i])
    except:
        num_error += 1
    '''    
    if(num_error>10):
        break
    '''
    final_train_data.append(temp_train)
    final_train_label.append(temp_label)
    
#final_train_data = np.asarray(final_train_data)
#final_train_label = np.asarray(final_train_label)
print('number of error is : ', num_error)
print('total data is : ', len(final_train_data))
with open('final_train_data.pickle', 'wb') as f:
    pickle.dump(final_train_data, f, pickle.HIGHEST_PROTOCOL)
    
with open('final_train_label.pickle', 'wb') as f:
    pickle.dump(final_train_label, f, pickle.HIGHEST_PROTOCOL)

making data.... 0


NameError: name 'temp_train' is not defined

In [79]:
len(train_data_preprocess)

826199

In [80]:
import numpy as np
import tensorflow as tf
data_num = 826874
train_embedding_dim= 584
item_embedding_dim = 157
latent_dim = 200
learning_rate = 1e-3
session_data = np.zeros((data_num,train_embedding_dim))
#positive_label = np.zeros((data_num, item_embedding_dim))
#negative_labels = np.zeros((data_num, item_embedding_dim))
#negative_number = np.zeros((data_num, 1))
item = np.zeros((3,4)) # 임시

In [81]:
tf.reset_default_graph()
session_feature = tf.get_variable("session_feature", [train_embedding_dim, latent_dim], initializer=tf.random_normal_initializer())
item_feature = tf.get_variable("item_feature", [item_embedding_dim, latent_dim], initializer=tf.random_normal_initializer())
session_bias = tf.get_variable("session_bias", [train_embedding_dim], initializer=tf.random_normal_initializer())
item_bias = tf.get_variable("item_bias", [item_embedding_dim], initializer=tf.random_normal_initializer())

In [82]:
train_data = tf.placeholder(tf.float32, shape=(train_embedding_dim))
train_items  = tf.placeholder(tf.float32, shape=(None, item_embedding_dim))
train_index = tf.placeholder(tf.int32, [1])

In [83]:
train_index[0]

<tf.Tensor 'strided_slice:0' shape=() dtype=int32>

In [84]:
print(tf.matmul(tf.matmul(train_items, item_feature), tf.transpose(tf.matmul(tf.expand_dims(train_data,0), session_feature))))
#x =  tf.matmul(tf.matmul(train_items, item_feature), tf.transpose(tf.matmul(tf.expand_dims(train_data,0), session_feature)))
print(tf.matmul(tf.expand_dims(train_data,0), tf.expand_dims(session_bias,1)))
print(tf.matmul(train_items, tf.expand_dims(item_bias,1)))

scores = tf.matmul(tf.matmul(train_items, item_feature), tf.transpose(tf.matmul(tf.expand_dims(train_data,0), session_feature))) +  \
tf.matmul(tf.expand_dims(train_data,0), tf.expand_dims(session_bias,1)) +\
tf.matmul(train_items, tf.expand_dims(item_bias,1))
#print(positive_score)

Tensor("MatMul_2:0", shape=(?, 1), dtype=float32)
Tensor("MatMul_3:0", shape=(1, 1), dtype=float32)
Tensor("MatMul_4:0", shape=(?, 1), dtype=float32)


In [85]:
loss = -tf.reduce_mean(tf.math.log(tf.sigmoid(scores[train_index[0]] - scores)), 0)

In [86]:
loss2 = tf.reduce_mean(tf.sigmoid(scores- scores[train_index[0]]) + tf.sigmoid(tf.math.pow(scores,2)))

In [87]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss2)

In [88]:
scores

<tf.Tensor 'add_1:0' shape=(?, 1) dtype=float32>

In [89]:
accuracy = tf.cast(tf.argmax(scores)[0] == train_index[0], tf.float32)

In [90]:
accuracy

<tf.Tensor 'Cast:0' shape=() dtype=float32>

In [158]:
batch_data, batch_items, correct_index = MakeEmbeddingVector3(train_data_preprocess[2])
batch_data  = np.asarray(batch_data)
batch_items = np.asarray(batch_items)

In [154]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
sc = sess.run(scores, feed_dict = {train_data : batch_data, train_items : batch_items, train_index : correct_index})
sc = sess.run(scores, feed_dict = {train_data : batch_data, train_items : batch_items, train_index : correct_index})
#_loss = sess.run(scores, feed_dict = {train_data : batch_data, train_items : batch_items})
#te = sess.run(temp_value, feed_dict = {train_data : batch_data, train_items : batch_items})

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [93]:
sess.close()

In [159]:
sc = sess.run(scores, feed_dict = {train_data : batch_data, train_items : batch_items, train_index : correct_index})

In [160]:
def get_MRS(score, idx):
    return (list(np.squeeze(np.argsort(score, axis = 0))).index(idx)+1)/np.shape(score)[0]

In [162]:
len(sc)

25

In [126]:
(list(np.squeeze(np.argsort(sc, axis = 0))).index(2)+1)/np.shape(sc)[0]

0.36

In [97]:
all_data_number = len(train_data_preprocess)
print(all_data_number)


826199


In [ ]:
final_train_data = train_data_preprocess[:810000]
final_valid_data = train_data_preprocess[810000:]
num_train = len(final_train_data)
num_valid = len(final_valid_data)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
max_mrs = 0
#all_data_number = len(train_data_preprocess)
saver = tf.train.Saver()
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    step = 0
    mean_loss = 0
    mean_rs = 0
    data_number = 0
    for i in range(num_train):
        #print(i)
        #if i>100:
        #    break
        
        
        batch_data, batch_items, correct_index = MakeEmbeddingVector3(final_train_data[i])
        batch_data  = np.asarray(batch_data)
        batch_items = np.asarray(batch_items)

        sess.run(optimizer, feed_dict = {train_data : batch_data, train_items : batch_items, train_index : correct_index})
        ls = sess.run(loss2, feed_dict = {train_data : batch_data, train_items : batch_items, train_index : correct_index})
        _score = sess.run(scores, feed_dict = {train_data : batch_data, train_items : batch_items, train_index : correct_index})
        if(len(_score) == 1):
            current_rs = 1
        else:
            current_rs = get_MRS(_score, correct_index[0])

            
        #print(correct_index)
        #print(_score)
        #print(np.argmax(_score, axis = 0)[0])
        #print(correct_index[0])

        mean_rs += current_rs 
        mean_loss += ls
        data_number +=1
        if (step % 500 == 1):
            #data_index = sess.run(tf.argmax(scores)[0], feed_dict = {train_data : batch_data, train_items : batch_items, train_index : correct_index})

            #acc = sess.run(accuracy, feed_dict = {train_data : batch_data, train_items : batch_items, train_index : correct_index})
            print('step', step )
            print("output index is : ", np.argmax(_score, axis = 0)[0])
            print("correct index is : ", correct_index[0])
            print("corrent rs is : ", current_rs)
            print("mean loss is : ", mean_loss / data_number)
            print("mean reciprocal score is : ", mean_rs / data_number)
            print("data num is : ", data_number)
            if(mean_rs / data_number > max_mrs):
                max_mrs = mean_rs /data_number
                print('saved')
                saver.save(sess, './saved_model/task1')
            mean_rs =0
            mean_loss = 0
            data_number = 0
            
        step +=1
                

        

step 1
output index is :  8
correct index is :  0
corrent rs is :  0.44
mean loss is :  1.6118367910385132
mean reciprocal score is :  0.4
data num is :  2
saved
step 501
output index is :  8
correct index is :  2
corrent rs is :  0.52
mean loss is :  1.4569673727750778
mean reciprocal score is :  0.5658128303953268
data num is :  500
saved
step 1001
output index is :  0
correct index is :  18
corrent rs is :  0.3181818181818182
mean loss is :  1.434384118080139
mean reciprocal score is :  0.5930667333947072
data num is :  500
saved
step 1501
output index is :  15
correct index is :  0
corrent rs is :  0.96
mean loss is :  1.4319894909858704
mean reciprocal score is :  0.5940990164382358
data num is :  500
saved
step 2001
output index is :  3
correct index is :  1
corrent rs is :  0.6875
mean loss is :  1.452891210794449
mean reciprocal score is :  0.5726041523805014
data num is :  500
step 2501
output index is :  5
correct index is :  3
corrent rs is :  0.6
mean loss is :  1.435054426

In [ ]:
def test():
    with tf.Session(config=config) as sess:
        sess.run(tf.global_variables_initializer())
        mean_loss = 0
        mean_rs = 0

        num_valid = len(final_valid_data)
        for i in range(num_valid):

                batch_data, batch_items, correct_index = MakeEmbeddingVector3(final_valid_data[i])
                batch_data  = np.asarray(batch_data)
                batch_items = np.asarray(batch_items)

                sess.run(optimizer, feed_dict = {train_data : batch_data, train_items : batch_items, train_index : correct_index})
                ls = sess.run(loss2, feed_dict = {train_data : batch_data, train_items : batch_items, train_index : correct_index})
                _score = sess.run(scores, feed_dict = {train_data : batch_data, train_items : batch_items, train_index : correct_index})
                if(len(_score) == 1):
                    current_rs = 1
                else:
                    current_rs = get_MRS(_score, correct_index[0])

                mean_rs += current_rs 
                mean_loss += ls
                data_number +=1
                
        print("mean loss is : ", mean_loss /num_valid)
        print("mean reciprocal score is : ", mear_rs /num_valid)
        return mean_loss /num_valid, mear_rs /num_valid            